# Monte Carlo 

In [1]:
def MonteCarloErrorAtArbitPoint(means, std_devs, deg = 1, sampling_size = 1000, x=0): 
    """
    Input: arrays means and standard deviations of the various scalings of noise, degree of fit, 
            sampling_size is how many times to sample from the Gaussian ,x is point at which to 
            approx error
    Assumption: Scaled by integer values ranging from 1 to X where X>1
    
    Output: Error estimate
    """
    n = len(means)
    
    #Sampling from Gaussian for each scaling 
    all_samples = []
    debug_means = []
    for i in range(n): 
        samples = np.random.normal(means[i], std_devs[i], sampling_size)
        all_samples.append(samples)
        #for debugging
        debug_means.append(means[i] - np.mean(samples))
    #print(debug_means)
    
    #Doing Extrapolation for all sampling_size samples (1000 if set to default)
    x_noise_energies = []
    for j in range(sampling_size): 
        energies = []
        for i in range(n): 
            energies.append(all_samples[i][j])
            
        
    ##Polynomial
        ####need the scale values: 
        scales = []
        for s in range(1, n+1): 
            scales.append(s)
        ####making the function 
        param = np.polyfit(scales, energies, deg)
        f = np.poly1d(param)
        ####extrapolation
        result = f(x)
        x_noise_energies.append(result)
            
    
    #Returning error i.e. std 
    error = np.std(x_noise_energies)
    return error

# Covariance

In [2]:
def ErrorOnPoint(func, cov, x): 
    """
    Input: Function, covariance matrix and x point to compute error at 
    Output: Error ---scalar value
    """
    #Gradient 
    degree = func.order
    grad = []
    for d in range(degree+1): 
        term = x**(degree-d)
        grad.append([term])
    #print('Gradient is : ')
    #print(grad)
    
    #Covariance Matrix Calc
    err_sq = (np.matmul(np.transpose(grad), np.matmul(cov, grad))).item()
    #print(err_sq)
    err = np.sqrt(err_sq)
    return err

# Monte Carlo for Richardson + Richardson Extrap

#### This version only calculates error at 0 point 

In [3]:
def MonteCarloError(means, std_devs, poly_fit = False, deg = 1, sampling_size = 1000): 
    """
    Input: arrays means and standard deviations of the various scalings of noise, whether to do richardson i.e. no poly fit 
    If doing polyfit, then specify degree, sampling_size is how many times to sample from the Gaussian 
    
    Assumption: Scaled by integer values ranging from 1 to X where X>1
    
    Output: Error estimate
    """
    n = len(means)
    
    #Sampling from Gaussian for each scaling 
    all_samples = []
    debug_means = []
    for i in range(n): 
        samples = np.random.normal(means[i], std_devs[i], sampling_size)
        all_samples.append(samples)
        #for debugging
        debug_means.append(means[i] - np.mean(samples))
    #print(debug_means)
    
    #Doing Extrapolation for all sampling_size samples (1000 if set to default)
    zero_noise_energies = []
    for j in range(sampling_size): 
        energies = []
        for i in range(n): 
            energies.append(all_samples[i][j])
            
        ##Doing either richardson or funcs 
        ###Polynomial
        if poly_fit: 
            ####need the scale values: 
            scales = []
            for s in range(1, n+1): 
                scales.append(s)
            ####making the function 
            param = np.polyfit(scales, energies, deg)
            f = np.poly1d(param)
            ####extrapolation
            result = f(0)
            zero_noise_energies.append(result)
            
        ###Richardson
        else: 
            result = RichardsonExtrap(energies)
            zero_noise_energies.append(result)
    
    #Returning error i.e. std 
    error = np.std(zero_noise_energies)
    return error
    

In [4]:
def RichardsonExtrap(energies): 
    """
    Input: An array of energies scaled with different stretch factors--normal integer multiples only --eg 1,2,3,4..
    Output: Zero Error extrapolated answer
    """
    n = len(energies)
    
    #setting up all equations 
    equations = []
    for eq_num in range(1,n+1): 
        equation = []
        if eq_num == 1: 
            for i in range(n): 
                equation.append(1)
            equations = [equation]
        else: 
            for term_num in range(1,n+1): 
                term = term_num**(eq_num-1)
                equation.append(term)
            equations = np.append(equations, [equation], axis = 0)
    #print(equations)
    
    #Now filling up equals to matrix 
    equals_to = [1]
    for i in range(1, n): 
        equals_to.append(0)
    #print(equals_to)
    
    #solving the system
    coeff = np.linalg.solve(equations, equals_to)
    #print(coeff)
    
    #Combine coeff with energies to get zero noise result 
    result = np.dot(coeff, energies)
    return result